In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


def transition_delay(t: int, *args):
    for a in args:
        a.update_layout(transition_duration=t)
        
# Download link via the below lik
#data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

data = pd.read_csv('historical_automobile_sales.csv')
app = dash.Dash(__name__)
#server = app.server
# Set the title of the dashboard
app.title = "Automobile Sales Statistics Dashboard"


year_list = [i for i in range(1980, 2024, 1)]

app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard",
            style={"text-align": "center","color": "#503D36", 'font-size': 24}),
    html.Div([
        html.Label("Select Statistics:"),
        
# Add First Dropdown
        dcc.Dropdown(
            id="dropdown-statistics",
            options=[
                {"label": "Yearly Statistics", "value": "Yearly Statistics"},
                {"label": "Recession Period Statistics", "value": "Recession Period Statistics"}
            ],
            value="Select Statistics",
            placeholder="Select a report type",
        )
    ], style={"margin": "10px 10px"}),
    
# Add Second Dropdown
    html.Div(dcc.Dropdown(
        id="select-year",
        options=[{'label': i, 'value': i} for i in year_list],
        value="",
        placeholder="Select Year"
    ), style={"margin": "10px 10px"}),
    
# Add a division for an output display    
    html.Div([html.Div(
        id="output-container",
        className="chart-grid",
        style={"display": "flex"}
    )])
])

#Callbacks
@app.callback(Output("select-year", "disabled"), Input("dropdown-statistics", "value"))
def update_dropdown(value):
    if value == "Yearly Statistics":
        return False
    return True


@app.callback(Output("output-container", "children"),
              [Input("dropdown-statistics", "value"), Input("select-year", "value")])
def update_plots(stat, year):
    if stat == "Recession Period Statistics":
        recession_df = data[data.Recession == 1]

#Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
         # grouping data for plotting        
        yearly_rec = recession_df.groupby("Year")["Automobile_Sales"].mean().reset_index()
        
         # Plotting the line graph
        c1 = dcc.Graph(figure=px.line(yearly_rec, x="Year", y="Automobile_Sales",
                                      title="Automobile sales fluctuate over Recession Period (year wise)"))
       
        #Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart

        vtype_rec = recession_df.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        c2 = dcc.Graph(figure=px.bar(vtype_rec, x="Vehicle_Type", y="Automobile_Sales",
                                     title="Average number of vehicles sold by vehicle type"))
        
        # Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
            # grouping data for plotting
        exp_rec = recession_df.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index()
        c3 = dcc.Graph(figure=px.pie(exp_rec, "Vehicle_Type", "Advertising_Expenditure",
                                     title="Total expenditure share by vehicle type during recessions"))
        
        # Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
        f_rec = recession_df.groupby(["unemployment_rate", "Vehicle_Type"], as_index=False)["Automobile_Sales"].sum()
        Veh_Types = recession_df.Vehicle_Type.unique()
        fig = go.Figure()
        for v in Veh_Types:
            fig.add_trace(go.Bar(x=f_rec[f_rec.Vehicle_Type == v]["unemployment_rate"],
                                 y=f_rec[f_rec.Vehicle_Type == v]["Automobile_Sales"], name=v))
        fig.update_layout(title="Effect of unemployment rate on vehicle type and sales",
                          xaxis_title="unemployment_rate", yaxis_title="Automobile_Sales")
        c4 = dcc.Graph(figure=fig)

        return [
            html.Div(className="chart-item", children=[html.Div(children=c1), html.Div(children=c2)]),
            html.Div(className="chart-item", children=[html.Div(children=c3), html.Div(children=c4)])
        ]
    
    # Yearly Statistic Report Plots                             
    elif str(year).isnumeric() and stat == "Yearly Statistics":
        year_data = data[data.Year == int(year)]

        # Plot 1 :Yearly Automobile sales using line chart for the whole period.

        yearly_sales = data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        f1 = px.line(yearly_sales, x="Year", y="Automobile_Sales",
                     title="Automobile sales (year wise)")

        # Plot 2 :Total Monthly Automobile sales using line chart.

        monthly_sales = year_data.groupby("Month")["Automobile_Sales"].mean().reset_index()
        f2 = px.line(monthly_sales, x="Month", y="Automobile_Sales",
                     title=f"Automobile sales (Year {year} months)")

        # Plot 3 bar chart for average number of vehicles sold during the given year

        vtypes_sales = year_data.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index()
        f3 = px.bar(vtypes_sales, x="Vehicle_Type", y="Automobile_Sales",
                    title=f"Average Vehicles Sold by Vehicle Type in the year {year}")

        # Plot 4 Total Advertisement Expenditure for each vehicle using pie chart

        exp_rec = year_data.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index()
        f4 = px.pie(exp_rec, "Vehicle_Type", "Advertising_Expenditure",
                    title=f"Total expenditure share by vehicle type in year {year}")

        transition_delay(1000, f1, f2, f3, f4)

        c1 = dcc.Graph(figure=f1)
        c2 = dcc.Graph(figure=f2)
        c3 = dcc.Graph(figure=f3)
        c4 = dcc.Graph(figure=f4)

        return [
            html.Div(className="chart-item", children=[html.Div(children=c1), html.Div(children=c2)]),
            html.Div(className="chart-item", children=[html.Div(children=c3), html.Div(children=c4)])
        ]


if __name__ == "__main__":
    app.run()